In [1]:
from keras.models  import load_model
import keras.backend as K

import cPickle
import sys
sys.path.insert(0, '../../preprocess')
import vectorizer

import pandas as pd
import numpy as np

aspects = ['population', 'intervention', 'outcome']

Using TensorFlow backend.


In [2]:
vec = cPickle.load(open('../data/vectorizers/allfields_with_embedding_5000.p', 'rb'))

In [3]:
index = vec.index
C = {'inputs' : ['abstract', 'population', 'intervention', 'outcome']}
for input in C['inputs'] :
    input_range = index[input]
    C[input] = vec.X[input_range[0]:input_range[1]]

In [80]:
model_pop = load_model('../store/weights/population_WedOct1115:34:362017_setup//0/0-loss.h5')
model_int = load_model('../store/weights/intervention_WedOct1115:34:362017_setup//0/0-loss.h5')
model_out = load_model('../store/weights/outcome_WedOct1115:34:362017_setup//0/0-loss.h5')
model = {'population' : model_pop, 'intervention' : model_int, 'outcome' : model_out}

In [58]:
model_pop = load_model('../store/weights/population_WedOct1114:21:052017_setup/0/0-loss.h5')
model_int = load_model('../store/weights/intervention_WedOct1114:21:052017_setup/0/0-loss.h5')
model_out = load_model('../store/weights/outcome_WedOct1114:21:052017_setup/0/0-loss.h5')
model = {'population' : model_pop, 'intervention' : model_int, 'outcome' : model_out}

In [64]:
model_pop = load_model('../store/weights/population_WedOct1123:04:542017_setup/0/0-loss.h5')
model_int = load_model('../store/weights/intervention_WedOct1123:04:542017_setup/0/0-loss.h5')
model_out = load_model('../store/weights/outcome_WedOct1123:04:542017_setup/0/0-loss.h5')
model = {'population' : model_pop, 'intervention' : model_int, 'outcome' : model_out}

In [81]:
model_c1 = {}
for aspect in aspects :
    model_c1[aspect] = K.function(model[aspect].get_layer('pool').inputs, 
                          [model[aspect].get_layer('pool').layers[2].output])

In [66]:
filters_pico = {}
sample_length = 5000
sample = np.random.choice(C['abstract'].shape[0], sample_length, replace=False)
for input in C['inputs'] :
    for aspect in aspects :
        filters_pico[(aspect, input)] = model_c1[aspect]([C[input][sample]])[0]

In [67]:
for key in sorted(filters_pico.keys()) :
    print key, (filters_pico[key] > 0).sum()/sample_length

('intervention', 'abstract') 96
('intervention', 'intervention') 78
('intervention', 'outcome') 5
('intervention', 'population') 10
('outcome', 'abstract') 61
('outcome', 'intervention') 3
('outcome', 'outcome') 37
('outcome', 'population') 8
('population', 'abstract') 68
('population', 'intervention') 7
('population', 'outcome') 6
('population', 'population') 200


In [68]:
activated_words = {}
for aspect in aspects :
    filters = filters_pico[(aspect, 'abstract')]
    word_activations = np.zeros((vec.vocab_size, filters.shape[2]))
    for i in range(filters.shape[2]) :
        filter_i = filters[:,:,i]
        word2filter = np.dstack((C['abstract'][sample], filter_i))
        filter_w2f = word2filter[word2filter[:,:,1] > 0.0]
        for word in filter_w2f :
            word_activations[int(word[0]), i] = word[1]
    activated_words[aspect] = word_activations

In [69]:
top_words_df = pd.DataFrame()
top_words_size = 40
for aspect in aspects :
    popwords = activated_words[aspect].sum(axis=1)
    sorted_idx = np.argsort(popwords)
    top_words = sorted_idx[-top_words_size:]
    top_words = pd.Series([v for k, v in vec.idx2word.items() if k in top_words])
    top_words_df[aspect] = top_words
top_words_df

,population,intervention,outcome
0,adenocarcinoma,artesunate,adhesion
1,alzheimer,bleomycin,anthropometric
2,antifungal,booklet,apgar
3,ards,chloroquine,bacteriological
4,av,cpm,cgi-i
5,cf,decisions,colonization
6,cirrhotic,discretion,crashes
7,comorbidity,etoposide,crying
8,comparability,exp,cure
9,country,fluorouracil,cured


<b>Evaluating Filters on Cohen Data</b>

In [76]:
df = pd.read_csv('../data/files/test_cohen_dedup.csv')
cohenvec = cPickle.load(open('../data/vectorizers/cohendata_dedup_5000.p', 'rb'))

In [82]:
filter_aspect = {}
for aspect in aspects :
    filter_aspect[aspect] = model_c1[aspect]([cohenvec.X])[0]

In [83]:
activated_words_cohen = {}
for aspect in aspects :
    filters = filter_aspect[aspect]
    word_activations = np.zeros((cohenvec.vocab_size, filters.shape[2]))
    for i in range(filters.shape[2]) :
        filter_i = filters[:,:,i]
        word2filter = np.dstack((cohenvec.X, filter_i))
        filter_w2f = word2filter[word2filter[:,:,1] > 0.0]
        for word in filter_w2f :
            word_activations[int(word[0]), i] = word[1]
    activated_words_cohen[aspect] = word_activations

In [86]:
top_words_df_cohen = pd.DataFrame()
top_words_size = 40
for aspect in aspects :
    popwords = activated_words_cohen[aspect].sum(axis=1)
    sorted_idx = np.argsort(popwords)
    top_words = sorted_idx[-top_words_size:]
    top_words = pd.Series([v for k, v in cohenvec.idx2word.items() if k in top_words])
    top_words_df_cohen[aspect] = top_words
top_words_df_cohen

,population,intervention,outcome
0,adenocarcinoma,atenolol,accuracy
1,alzheimer,behaviors,adhesion
2,ami,chlorpromazine,bmd
3,asa,cognitive-behavioral,colonization
4,asian,cortex,cure
5,australian,decanoate,cured
6,boys,diazepam,curve
7,british,dipyridamole,disappearance
8,burns,encouraged,eradication
9,cd,extract,falling
